<a href="https://colab.research.google.com/github/weilly0912/Core-ML/blob/main/Tensorflow_Lite_Conveter.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **TensorFlow Lite 模組轉換大解密**
[@PINTO 大神 文章](https://qiita.com/PINTO/items/865250ee23a15339d556#4-2-1-4-weight-quantization-from-saved_model-weight-only-quantization)

[@PINTO 大神 Source Code](https://github.com/PINTO0309/PINTO_model_zoo)




## **0.Tensorflow Version 切換**

In [ ]:
# 切換版本
%tensorflow_version 1.x
import tensorflow
print(tensorflow.__version__)

TensorFlow 1.x selected.
1.15.2


## **1.Tensorflow 1.x / 2.x Model 應用方式**

*   (1) *ckpt to TensorFlow Lite ( Tensorflow 1.x version )*

*   (2) *freezen graph to TensorFlow Lite ( Tensorflow 1.x version )*

*   (3) *Savemodel to TensorFlow Lite   ( Tensorflow 2.x version)*

*   (4) *Example ( Tensorflow 1.x version )*

       4.1 重新儲存 ckpt

       4.2 ckpt 轉成 freezen graph

       4.3 freezen graph 轉成 saved_model
      
       4.4 轉成 TensorFlow Lite

*   (5) *ReBuild-Inupt Savemodel to TensorFlow Lite*


###**(1) ckpt to TensorFlow Lite**

In [ ]:
# Tensorflow 1.x 
import tensorflow as tf
from src import transform # 讀取模組架構, https://github.com/TurboCome/Style-transfer-of-picture/blob/master/src/transform.py
g = tf.compat.v1.Graph()
soft_config = tf.compat.v1.ConfigProto(allow_soft_placement=True)
soft_config.gpu_options.allow_growth = True
with g.as_default(), tf.compat.v1.Session(config=soft_config) as sess:
  img_placeholder = tf.compat.v1.placeholder(tf.float32, shape=[1, 474, 712, 3], name='img_placeholder') #輸入端節點(Netron 查看)
  preds = transform.net(img_placeholder) # 輸出端節點 ( Netron 查看 )
  saver = tf.compat.v1.train.Saver()
  saver.restore(sess, "model.ckpt" ) # ckpt file
  converter = tf.compat.v1.lite.TFLiteConverter.from_session(sess, [img_placeholder], [preds]) 
  converter.optimizations = [tf.lite.Optimize.DEFAULT]
  tflite_model = converter.convert()
  with tf.io.gfile.GFile( "model.tflite", 'wb') as f:
    f.write(tflite_model)
  print("Integer Quantization complete! -model.tflite")

###**(2) freezen graph to TensorFlow Lite**


In [ ]:
import tensorflow as tf
tf.compat.v1.enable_eager_execution()
# Weight Quantization - Input/Output=float32
graph_def_file="tflite_graph_with_postprocess.pb"
input_arrays=["normalized_input_image_tensor"]
output_arrays=['TFLite_Detection_PostProcess','TFLite_Detection_PostProcess:1', 
               'TFLite_Detection_PostProcess:2','TFLite_Detection_PostProcess:3']
input_tensor={"normalized_input_image_tensor":[1,300,300,3]}
converter = tf.lite.TFLiteConverter.from_frozen_graph(graph_def_file, input_arrays,output_arrays,input_tensor)
converter.optimizations = [tf.lite.Optimize.OPTIMIZE_FOR_SIZE]
converter.allow_custom_ops = True
tflite_quant_model = converter.convert()
with open('./ssdlite_mobilenet_v2_voc_300_weight_quant.tflite', 'wb') as w:
    w.write(tflite_quant_model)

###**(3) Savemodel to TensorFlow Lite**

In [ ]:
import tensorflow as tf
converter = tf.lite.TFLiteConverter.from_saved_model( "savedmodel" ) 
converter.optimizations = [tf.lite.Optimize.DEFAULT]
tflite_model = converter.convert()
with tf.io.gfile.GFile( "model.tflite" , 'wb') as f:
   f.write(tflite_model)
print("Quantization complete! - model.tflite")

###**(4) Example : 以範例 Cartoonize 介紹**

In [ ]:
# Dowload API
%cd /root
!git clone https://github.com/SystemErrorWang/White-box-Cartoonization

# 須修改與執行檔案兩次，修改內容依下
%cd /root/White-box-Cartoonization/test_code
!python3 cartoonize.py

In [ ]:
#需要更改 Cartoonize 
#須從代碼找到輸入端與輸出端資訊

# 4.1 重新輸出儲存檔案 ckpt
# checkpoint & **.index & **.meta
def cartoonize(load_folder, save_folder, model_path):
    import sys
    import shutil
    shutil.rmtree('./export', ignore_errors=True)
    input_photo = tf.placeholder(tf.float32, [1, 720, 720, 3], name='input') #<-- here
    network_out = network.unet_generator(input_photo)# 須加入架構
    final_out = guided_filter.guided_filter(input_photo, network_out, r=1, eps=5e-3)
    print("input_photo.name =", input_photo.name)
    print("input_photo.shape =", input_photo.shape)
    print("final_out.name =", final_out.name)
    print("final_out.shape =", final_out.shape)
    all_vars = tf.trainable_variables()
    gene_vars = [var for var in all_vars if 'generator' in var.name]
    saver = tf.train.Saver(var_list=gene_vars)
    config = tf.ConfigProto()
    config.gpu_options.allow_growth = True
    sess = tf.Session(config=config)
    sess.run(tf.global_variables_initializer())
    saver.restore(sess, tf.train.latest_checkpoint(model_path)) #/root/White-box-Cartoonization/test_code/saved_models
    saver.save(sess, './export/model.ckpt') #<-- here
    sys.exit(0) #<-- here & delete below

# 4.2 重新轉換為 Freeze_Graph
# **.pbtxt and  **.pb
def cartoonize(load_folder, save_folder, model_path):
    import sys
    graph = tf.get_default_graph()
    sess = tf.Session()
    saver = tf.train.import_meta_graph('./export/model.ckpt.meta')
    saver.restore(sess, './export/model.ckpt')
    tf.train.write_graph(sess.graph_def, './export', 'white_box_cartoonization_freeze_graph.pbtxt', as_text=True)
    tf.train.write_graph(sess.graph_def, './export', 'white_box_cartoonization_freeze_graph.pb', as_text=False)
    sys.exit(0)

# 4.3 Freeze_Graph to Saved_model
# TF 1.x 適用
import tensorflow as tf
import os
import shutil
from tensorflow.python import ops

def get_graph_def_from_file(graph_filepath):
  tf.compat.v1.reset_default_graph()
  with ops.Graph().as_default():
    with tf.compat.v1.gfile.GFile(graph_filepath, 'rb') as f:
      graph_def = tf.compat.v1.GraphDef()
      graph_def.ParseFromString(f.read())
      return graph_def

def convert_graph_def_to_saved_model(export_dir, graph_filepath, input_name, outputs):
  graph_def = get_graph_def_from_file(graph_filepath)
  with tf.compat.v1.Session(graph=tf.Graph()) as session:
    tf.import_graph_def(graph_def, name='')
    tf.compat.v1.saved_model.simple_save(
        session,
        export_dir,# change input_image to node.name if you know the name
        inputs={input_name: session.graph.get_tensor_by_name('{}:0'.format(node.name))
            for node in graph_def.node if node.op=='Placeholder'},
        outputs={t.rstrip(":0"):session.graph.get_tensor_by_name(t) for t in outputs}
    )
    print('Graph converted to SavedModel!')

#tf.compat.v1.enable_eager_execution()
input_name="input"  #here 
outputs = ['add_1:0'] #here
shutil.rmtree('./saved_model', ignore_errors=True)
convert_graph_def_to_saved_model('/root/White-box-Cartoonization/test_code/export/saved_model', '/root/White-box-Cartoonization/test_code/export/white_box_cartoonization_freeze_graph.pb', input_name, outputs)


In [ ]:
#重啟環境 解決tensorflow版本問題
import os
os.kill(os.getpid(), 9)

In [ ]:
!pip uninstall tensorflow -y
!pip install tensorflow==2.2.0

In [ ]:
# 4.4 Tensorflow Lite Converter
# Error : 無法順利轉換
%tensorflow_version 2.x
import tensorflow as tf
print(tf.__version__)
tf.compat.v1.enable_eager_execution()

# Weight Quantization - Input/Output=float32
converter = tf.lite.TFLiteConverter.from_saved_model('/root/White-box-Cartoonization/test_code/export/saved_model')

#converter.experimental_new_converter = True   #<--- Not necessary if you are using Tensorflow v2.2.x or later.
#converter.optimizations = [tf.lite.Optimize.OPTIMIZE_FOR_SIZE]
tflite_quant_model = converter.convert()
#with open('/root/White-box-Cartoonization/test_code/export/saved_model/white_box_cartoonization_weight_quant.tflite', 'wb') as w:
#    w.write(tflite_quant_model)
#print("Weight Quantization complete! - white_box_cartoonization_weight_quant.tflite")

#整數的話有配合 tfds.load 
#文章後續有介紹 整數量化 / 全整數量化 / 半浮點量化用法
#若要使用 tpu 請使用 edgetpu_compiler 再次生成 **_tpu.tflite

###**(5) ReBuild-Inupt Savemodel to TensorFlow Lite**

In [ ]:
#(尚未實現), Tensorflow JS 將會介紹
import tensorflow as tf
from tensorflow.tools.graph_transforms import TransformGraph

with tf.compat.v1.Session() as sess:

    # shape=[1, ?, ?, 3] -> shape=[1, 513, 513, 3]
    # name='image' specifies the placeholder name of the converted model
    inputs = tf.compat.v1.placeholder(tf.float32, shape=[1, 513, 513, 3], name='image')
    with tf.io.gfile.GFile('./model-mobilenet_v1_101.pb', 'rb') as f:
        graph_def = tf.compat.v1.GraphDef()
    graph_def.ParseFromString(f.read())

    # 'image:0' specifies the placeholder name of the model before conversion
    tf.graph_util.import_graph_def(graph_def, input_map={'image:0': inputs}, name='')
    print([n for n in tf.compat.v1.get_default_graph().as_graph_def().node if n.name == 'image'])

    # Delete Placeholder "image" before conversion
    # see: https://github.com/tensorflow/tensorflow/tree/master/tensorflow/tools/graph_transforms
    # TransformGraph(
    #     graph_def(),
    #     input_op_name,
    #     output_op_names,
    #     conversion options
    # )
    optimized_graph_def = TransformGraph(
                              tf.compat.v1.get_default_graph().as_graph_def(),
                              'image',
                              ['heatmap','offset_2','displacement_fwd_2','displacement_bwd_2'],
                              ['strip_unused_nodes(type=float, shape="1,513,513,3")'])

    tf.io.write_graph(optimized_graph_def, './', 'model-mobilenet_v1_101_513.pb', as_text=False)

## **[2.Tensorflow Hub 應用方式](https://colab.research.google.com/github/frogermcs/TFLite-Tester/blob/master/notebooks/Testing_TFLite_model.ipynb#scrollTo=9yM9UeUnF8t7)**

*   (1) Example : 以範例 Faster RCNN (object detection) 介紹

    1.1 載入 TF Hub 模組

    1.2 利用 Concrete Func 進行轉換為 tflite

### **(1) Example : 以範例 Faster RCNN 介紹**

In [ ]:
!pip install tensorflow_hub

In [ ]:
# faster rcnn
import tensorflow_hub as hub
hub_model = hub.load('https://tfhub.dev/tensorflow/faster_rcnn/inception_resnet_v2_640x640/1')

import tensorflow as tf
run_model = tf.function(lambda x : hub_model(x))
concrete_func = run_model.get_concrete_function(tf.TensorSpec(shape=[1,None,None,3], dtype=tf.uint8))

# Convert the model
converter = tf.lite.TFLiteConverter.from_concrete_functions([concrete_func])
converter.optimizations = [tf.lite.Optimize.DEFAULT]
converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS]
tflite_quant_model = converter.convert()
with open('/root/faster_rcnn_v2_640.tflite', 'wb') as w:
    w.write(tflite_quant_model)
print("Integer Quantization complete! - weights_integer_quant.tflite")

In [ ]:
# Load the model.
model = hub.load('https://tfhub.dev/google/yamnet/1')

# Tensorlfow Hub to TFLite
run_model = tf.function(lambda x : model(x))
concrete_func = run_model.get_concrete_function(tf.TensorSpec(shape=[None], dtype=tf.float32))

# Convert the model
converter = tf.lite.TFLiteConverter.from_concrete_functions([concrete_func])
converter.optimizations = [tf.lite.Optimize.DEFAULT]
converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS]
tflite_quant_model = converter.convert()
with open('/root/yamnet.tflite', 'wb') as w:
    w.write(tflite_quant_model)
print("Integer Quantization complete! - weights_integer_quant.tflite")

## **3.Tensorflow keras 1.x / 2.x 應用方式**


*   *(1) Keras 2.x to TensorFlow Lite*

*   *(2) Keras 2.x to TensorFlow Lite*

*   *(3) Exaple : 以範例 Faster Grad CAM 介紹*

      - 載入 json 架構 與 weight.h5 轉成 tflite (i.MX8MP平台可使用)

*   *(4) Extra : ReBuild-Inupt Keras to TensorFlow Lite*






### **(1) Keras 1.x to TensorFlow Lite**

In [ ]:
import tensorflow as tf
converter = tf.compat.v1.lite.TFLiteConverter.from_keras_model_file( 'model.h5' )
converter.optimizations = [tf.lite.Optimize.DEFAULT]
tflite_model = converter.convert()
with tf.io.gfile.GFile( "model.tflite" , 'wb') as f:
   f.write(tflite_model)
print("Quantization complete! - model.tflite ")


### **(2) Keras 2.x to TensorFlow Lite**

In [ ]:
import tensorflow as tf
tf.compat.v1.enable_eager_execution()
model = tf.keras.models.model_from_json(open( "model.json" ).read()) 
model.load_weights('weights.h5')
converter = tf.lite.TFLiteConverter.from_keras_model(model)
converter.optimizations = [tf.lite.Optimize.DEFAULT]
tflite_model = converter.convert()
with tf.io.gfile.GFile( "model.tflite" , 'wb') as f:
   f.write(tflite_model)
print("Quantization complete! - model.tflite ")

### **(3) Exaple : 以範例 Faster Grad CAM 介紹**

In [ ]:
%cd /root
!git clone https://github.com/shinmura0/Faster-Grad-CAM.git

In [ ]:
# TF Lite 量化(一般)
import tensorflow as tf
tf.compat.v1.enable_eager_execution()
model = tf.keras.models.model_from_json(open('/root/Faster-Grad-CAM/model/model.json').read())
model.load_weights('/root/Faster-Grad-CAM/model/weights.h5')
converter = tf.lite.TFLiteConverter.from_keras_model(model)
converter.optimizations = [tf.lite.Optimize.OPTIMIZE_FOR_SIZE]
tflite_quant_model = converter.convert()
with open('/root/Faster-Grad-CAM/model/weights_weight_quant.tflite', 'wb') as w:
    w.write(tflite_quant_model)
print("Weight Quantization complete! - weights_weight_quant.tflite")

In [ ]:
# TF Lite 量化(全整數-生產數據集)
%cd /root/Faster-Grad-CAM
!git clone https://github.com/karaage0703/janken_dataset.git

# 進入該資料夾
%cd /root/Faster-Grad-CAM/janken_dataset/gu

# 生成 dataset
from PIL import Image
import os, glob
import numpy as np
dataset = []
files = glob.glob("*.JPG")
for file in files:
    image = Image.open(file)
    image = image.convert("RGB")
    data = np.asarray(image)
    dataset.append(data)

dataset = np.array(dataset)
np.save("janken_dataset", dataset) #gererate **.npy


In [ ]:
# TF Lite 量化(全整數)
import tensorflow as tf
import numpy as np

def representative_dataset_gen():
    raw_test_data = np.load('/root/Faster-Grad-CAM/janken_dataset/gu/janken_dataset.npy')
    for image in raw_test_data:
        image = tf.image.resize(image, (96, 96))
        image = image / 255
        calibration_data = image[np.newaxis, :, :, :]
        yield [calibration_data]

tf.compat.v1.enable_eager_execution()

# Integer Quantization - Input/Output=float32
# INPUT  = input_1 (float32, 1 x 96 x 96 x 3)
# OUTPUT = block_16_expand_relu, global_average_pooling2d_1

model = tf.keras.models.model_from_json(open('/root/Faster-Grad-CAM/model/model.json').read())
model.load_weights('/root/Faster-Grad-CAM/model/weights.h5')
converter = tf.lite.TFLiteConverter.from_keras_model(model)
converter.optimizations = [tf.lite.Optimize.DEFAULT]
converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS]
converter.inference_input_type = tf.uint8
converter.inference_output_type = tf.uint8
converter.representative_dataset = representative_dataset_gen
tflite_quant_model = converter.convert()
with open('/root/Faster-Grad-CAM/model/weights_full_integer_quant.tflite', 'wb') as w:
    w.write(tflite_quant_model)
print("Integer Quantization complete! - weights_integer_quant.tflite")

#文章後續有介紹 整數量化 / 全整數量化 / 半浮點量化用法
#若要使用 tpu 請使用 edgetpu_compiler 再次生成 **_tpu.tflite

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/utils/generic_utils.py:497: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  category=CustomMaskWarning)


INFO:tensorflow:Assets written to: /tmp/tmpytkrb_3o/assets


INFO:tensorflow:Assets written to: /tmp/tmpytkrb_3o/assets


Integer Quantization complete! - weights_integer_quant.tflite


### **(4) Extra : ReBuild-Inupt Keras to TensorFlow Lite**

In [ ]:

# Method  : rebuild and reload weight -> set new input -> save
from keras.layers import Input, Flatten, Dense, Dropout, BatchNormalization
from keras.models import Model
from keras.models import Sequential
from keras.applications.vgg16 import VGG16

input = Input(shape=(480, 720, 3), name='image_input')

#re-model
initial_model = VGG16(weights='imagenet', include_top=False)
for layer in initial_model.layers:
    layer.trainable = False

x = Flatten()(initial_model(input))
x = Dense(1000, activation='relu')(x)
x = BatchNormalization()(x)
x = Dropout(0.5)(x)
x = Dense(1000, activation='relu')(x)
x = BatchNormalization()(x)
x = Dropout(0.5)(x)
x = Dense(14, activation='linear')(x)

model = Model(inputs=input, outputs=x)
model.load_weights('my_model_name.h5')

"""
inputs2 = Input((512, 512, 3))
...
x_new
model2 = Model(inputs=[inputs2], outputs=[x_new])
model2.compile(optimizer='adam', loss='mean_squared_error')
model2.set_weights(model.get_weights()) #set old model weitght
"""

## **4.Tensorflow Java Script 應用方式**

*   (1) Java Script to TensorFlow Lite

*   (2) Example : 以範例 PoseNet 介紹

    2.1 安裝 tfjs-graph-converter

    2.2 介紹 savemodel reinput 用法 (尚未試成功)


### **(1) Java Script to TensorFlow Lite**

In [ ]:
# Json to savedmodel
import tfjs_graph_converter.api as tfjs
tfjs.graph_model_to_saved_model( "model.json" , "realsavedmodel" )

In [ ]:
# TensorFlow Lite Converter
import tensorflow as tf
converter = tf.lite.TFLiteConverter.from_saved_model( "realsavedmodel" ) 
converter.optimizations = [tf.lite.Optimize.DEFAULT]
tflite_model = converter.convert()
with tf.io.gfile.GFile( "model.tflite" , 'wb') as f:
   f.write(tflite_model)
print("Quantization complete! - model.tflite ")

### **(2) Example : 以範例 PoseNet 介紹**

In [ ]:
!pip install tfjs-graph-converter

In [ ]:
#下載 posenet API
%cd /root
!git clone https://github.com/atomicbits/posenet-python.git

#下載測試圖檔
%cd /root/posenet-python
!mkdir images
%cd images
!wget https://i2.kknews.cc/SIG=l6lv5l/46sn0003ss48o8769p27.jpg

In [ ]:
# 額外方法 weilly
%cd /root/posenet-python
!mkdir model
%cd model
!wget https://storage.googleapis.com/tfjs-models/savedmodel/posenet/mobilenet/quant1/075/model-stride8.json
!wget https://storage.googleapis.com/tfjs-models/savedmodel/posenet/mobilenet/quant1/075/model-stride16.json
!wget https://storage.googleapis.com/tfjs-models/savedmodel/posenet/mobilenet/quant1/075/group1-shard1of1.bin

import tfjs_graph_converter.api as tfjs
tfjs.graph_model_to_saved_model("/root/posenet-python/model/model-stride16.json","mobilenet")

In [ ]:
%cd /root/posenet-python
!python3 image_demo.py --model resnet50 --stride 16 --image_dir ./images --output_dir ./output

In [ ]:
#重啟環境 解決tensorflow版本問題
import os
os.kill(os.getpid(), 9)

In [ ]:
%tensorflow_version 1.x
import tensorflow as tf
print(tf.__version__)

TensorFlow 1.x selected.
1.15.2


In [ ]:
### re-input and re-saved model
# Error : ValueError: NodeDef mentions attr 'explicit_paddings' not in Op<
import sys
import tensorflow as tf
from tensorflow.tools.graph_transforms import TransformGraph
from tensorflow.python.platform import gfile
from tensorflow.core.protobuf import saved_model_pb2
from tensorflow.python.util import compat

with tf.compat.v1.Session() as sess:

    # shape=[1, ?, ?, 3] -> shape=[1, 513, 513, 3]
    # name='image' specifies the placeholder name of the converted model

    inputs = tf.compat.v1.placeholder(tf.float32, shape=[1, 513, 513, 3], name='image')
    #inputs = tf.compat.v1.placeholder(tf.float32, shape=[1, 385, 385, 3], name='image')
    #inputs = tf.compat.v1.placeholder(tf.float32, shape=[1, 321, 321, 3], name='image')
    #inputs = tf.compat.v1.placeholder(tf.float32, shape=[1, 257, 257, 3], name='image')
    #inputs = tf.compat.v1.placeholder(tf.float32, shape=[1, 225, 225, 3], name='image')

    with gfile.FastGFile('/root/posenet-python/_tf_models/posenet/resnet50_float/stride16/saved_model.pb', 'rb') as f:
        data = compat.as_bytes(f.read())
        sm = saved_model_pb2.SavedModel()
        sm.ParseFromString(data)
        if 1 != len(sm.meta_graphs):
            print('More than one graph found. Not sure which to write')
            sys.exit(1)

    # 'image:0' specifies the placeholder name of the model before conversion
    tf.graph_util.import_graph_def(sm.meta_graphs[0].graph_def, input_map={'sub_2:0': inputs}, name='')
    print([n for n in tf.compat.v1.get_default_graph().as_graph_def().node if n.name == 'image'])

    # Delete Placeholder "image" before conversion
    # see: https://github.com/tensorflow/tensorflow/tree/master/tensorflow/tools/graph_transforms
    # TransformGraph(
    #     graph_def(),
    #     input_name,
    #     output_names,
    #     conversion options
    # )
    optimized_graph_def = TransformGraph(tf.compat.v1.get_default_graph().as_graph_def(),
                        'image',
                        ['float_heatmaps','float_short_offsets','resnet_v1_50/displacement_fwd_2/BiasAdd','resnet_v1_50/displacement_bwd_2/BiasAdd'],
                         ['strip_unused_nodes(type=float, shape="1,513,513,3")'])

    tf.io.write_graph(optimized_graph_def, './', 'posenet_resnet50_32_513.pb', as_text=False)

#文章後續有介紹 整數量化 / 全整數量化 / 半浮點量化用法
#若要使用 tpu 請使用 edgetpu_compiler 再次生成 **_tpu.tflite

In [ ]:
### re-input and re-saved model (Some_changes_from_TFv2.x_to_TFv1.x_to_import_into_saved_model)
# Error : ValueError: NodeDef mentions attr 'explicit_paddings' not in Op<name=MaxPool
import sys
import tensorflow as tf
from tensorflow.python.platform import gfile
from tensorflow.core.protobuf import saved_model_pb2
from tensorflow.python.util import compat
from tensorflow.tools.graph_transforms import TransformGraph

with tf.compat.v1.Session() as sess:

    # shape=[1, ?, ?, 3] -> shape=[1, 513, 513, 3]
    # name='image' specifies the placeholder name of the converted model

    inputs = tf.compat.v1.placeholder(tf.float32, shape=[1, 224, 224, 3], name='image')
    #inputs = tf.compat.v1.placeholder(tf.float32, shape=[1, 385, 385, 3], name='image')
    #inputs = tf.compat.v1.placeholder(tf.float32, shape=[1, 321, 321, 3], name='image')
    #inputs = tf.compat.v1.placeholder(tf.float32, shape=[1, 257, 257, 3], name='image')
    #inputs = tf.compat.v1.placeholder(tf.float32, shape=[1, 225, 225, 3], name='image')

    #Some_changes_from_TFv2.x_to_TFv1.x_to_import_into_saved_model
    with gfile.FastGFile('/root/posenet-python/model/mobilenet/saved_model.pb', 'rb') as f:
      data = compat.as_bytes(f.read())
      sm = saved_model_pb2.SavedModel()
      sm.ParseFromString(data)
      if 1 != len(sm.meta_graphs):
        print('More than one graph found. Not sure which to write')
        sys.exit(1)

    # 'image:0' specifies the placeholder name of the model before conversion
    tf.graph_util.import_graph_def(sm.meta_graphs[0].graph_def, input_map={'sub_2:0': inputs}, name='')
    print([n for n in tf.compat.v1.get_default_graph().as_graph_def().node if n.name == 'image'])

    # Delete Placeholder "image" before conversion
    # see: https://github.com/tensorflow/tensorflow/tree/master/tensorflow/tools/graph_transforms
    # TransformGraph(
    #     graph_def(),
    #     input_name,
    #     output_names,
    #     conversion options
    # )
    optimized_graph_def = TransformGraph(tf.compat.v1.get_default_graph().as_graph_def(),
                        'image',
                        ['float_heatmaps','float_short_offsets','resnet_v1_50/displacement_fwd_2/BiasAdd','resnet_v1_50/displacement_bwd_2/BiasAdd'],
                         ['strip_unused_nodes(type=float, shape="1,224,224,3")'])

    tf.io.write_graph(optimized_graph_def, './', 'posenet_mobilnet_224.pb', as_text=False)

## **5.Tensorflow Slim 應用方式**
**量化包含 Tensorflow Lite 不支持但 Tensorflow 支持的操作的模型 (能使用)**
*   (1) Example : 以範例 Mask R-CNN for Object Detection and Segmentation 介紹
      
      1.1 Flex Delegate 試用 




### **(1) Example : 以範例 Mask R-CNN 介紹**

In [ ]:
#重啟環境 解決tensorflow版本問題
import os
os.kill(os.getpid(), 9)

In [ ]:
# 下載 tfslim / ensorflowLite-flexdelegate / Mask_RCNN

%tensorflow_version 1.x
import tensorflow as tf
print(tf.__version__)

%cd /root
!git clone https://github.com/tensorflow/models.git 
!git clone https://github.com/matterport/Mask_RCNN/
!git clone https://github.com/PINTO0309/TensorflowLite-flexdelegate
!wget http://download.tensorflow.org/models/object_detection/mask_rcnn_inception_v2_coco_2018_01_28.tar.gz
!tar -zxvf mask_rcnn_inception_v2_coco_2018_01_28.tar.gz
!rm mask_rcnn_inception_v2_coco_2018_01_28.tar.gz
!mkdir -p /root/mask_rcnn_inception_v2_coco_2018_01_28/export #生成檔案

In [ ]:
#安裝與下載 tf slim
%cd /root
#git clone https://github.com/tensorflow/models.git 

%cd /root/models/research
!protoc object_detection/protos/*.proto --python_out=. # gernate *.proto
!python setup.py build

import os
os.environ['PYTHONPATH'] += ':/root/models/research/:/root/models/research/slim/:/root/models/research/object_detection/utils/:/root/models/research/object_detection'
!pip install tf_slim
!python object_detection/builders/model_builder_test.py

!export PYTHONPATH=`pwd`:`pwd`/slim:$PYTHONPATH

In [ ]:
#生成 freezon graph
!python3 /root/models/research/object_detection/export_inference_graph.py \
  --input_type=image_tensor \
  --pipeline_config_path=/root/mask_rcnn_inception_v2_coco_2018_01_28/pipeline.config \
  --trained_checkpoint_prefix=/root/mask_rcnn_inception_v2_coco_2018_01_28/model.ckpt \
  --output_directory=/root/mask_rcnn_inception_v2_coco_2018_01_28/test \
  --input_shape=1,256,256,3 \
  --write_inference_graph=True

In [ ]:
!python3 object_detection/export_inference_graph.py \
  --input_type=image_tensor \
  --pipeline_config_path=/root/mask_rcnn_inception_v2_coco_2018_01_28/pipeline.config \
  --trained_checkpoint_prefix=/root/mask_rcnn_inception_v2_coco_2018_01_28/model.ckpt \
  --output_directory=/root/mask_rcnn_inception_v2_coco_2018_01_28/test \
  --input_shape=1,512,512,3 \
  --write_inference_graph=True

In [ ]:
#重啟環境 解決tensorflow版本問題
import os
os.kill(os.getpid(), 9)

In [ ]:
# Tensorflow Lite 轉換
# tensorflow==2.2.0 (該範例無法整數)
%tensorflow_version 2.x
import tensorflow as tf
import numpy as np

# Weight Quantization - Input/Output=float32
converter = tf.lite.TFLiteConverter.from_saved_model('/root/mask_rcnn_inception_v2_coco_2018_01_28/test/saved_model')
converter.optimizations = [tf.lite.Optimize.OPTIMIZE_FOR_SIZE]
converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS,tf.lite.OpsSet.SELECT_TF_OPS]
tflite_quant_model = converter.convert()
with open('/root/mask_rcnn_inception_v2_coco_2018_01_28/test/mask_rcnn_inception_v2_coco_weight_quant.tflite', 'wb') as w:
    w.write(tflite_quant_model)
print("Weight Quantization complete! - mask_rcnn_inception_v2_coco_weight_quant.tflite")
#文章後續有介紹 整數量化 / 全整數量化 / 半浮點量化用法
#若要使用 tpu 請使用 edgetpu_compiler 再次生成 **_tpu.tflite

INFO:tensorflow:Saver not created because there are no variables in the graph to restore
INFO:tensorflow:Saver not created because there are no variables in the graph to restore
Weight Quantization complete! - mask_rcnn_inception_v2_coco_weight_quant.tflite


## **6.ONNX/OpenVINO Model 應用**


*   (1) ONNX to TensorFlow Lite

*   (2) [Example : 以 3D Multi-Person Pose Estimation 介紹](https://zhuanlan.zhihu.com/p/370398974)

    2.1 安裝 openvino model and tool , onnx2keras

    2.2 從 OpenVINO 下載 onnx 模組的方法

    2.3 轉換 3D Multi-Person Pose TF Lite 範例




###**(1) ONNX to TensorFlow Lite**

In [ ]:
# onnx-tf install
! pip install onnx-tf

In [ ]:
# onnx to tensorflow
! onnx-tf convert -i /root/facemesh.onnx -o /root/facemesh

In [ ]:
import tensorflow as tf
converter = tf.lite.TFLiteConverter.from_saved_model( "savedmodel" )
converter.optimizations = [tf.lite.Optimize.DEFAULT]
tflite_model = converter.convert()
with tf.io.gfile.GFile( "model.tflite” , 'wb') as f:
   f.write(tflite_model)
print("Quantization complete! - model.tflite ")

###**(2) Example : 以 3D Multi-Person Pose Estimation 介紹**

In [ ]:
#下載 API
#https://docs.openvinotoolkit.org/2020.2/_tools_downloader_README.html
%cd /root
!git clone https://github.com/opencv/open_model_zoo.git #OpenVINO™ Toolkit - Open Model Zoo repository

#安裝 openvino dev tool
#https://github.com/openvinotoolkit/openvino
!pip install openvino-dev
!git clone https://github.com/openvinotoolkit/openvino

#安裝 ONNX 2 Keras
!pip install onnx2keras

#安裝 ONNX 2 tensorflow (weilly建議)
%cd /root
!git clone https://github.com/onnx/onnx-tensorflow.git 
%cd onnx-tensorflow
!pip install -e .

# 下載模組必要套件
%cd /root/open_model_zoo/tools/downloader
!python -mpip install --user -r ./requirements.in
!python -mpip install --user -r ./requirements-tensorflow.in


In [ ]:
# 下載模組
%cd /root/open_model_zoo/tools/downloader
!python downloader.py --name human-pose-estimation-3d-0001 --precisions FP32
!./converter.py --mo /root/openvino_env/openvino/model-optimizer/mo.py --name human-pose-estimation-3d-0001
#here -> /root/open_model_zoo/tools/downloader/public/human-pose-estimation-3d-0001

In [ ]:
# onnx 轉成 saveedmodel -方法1
import onnx
from onnx2keras import onnx_to_keras
import tensorflow as tf
import shutil

onnx_model = onnx.load('/root/open_model_zoo/tools/downloader/public/human-pose-estimation-3d-0001/human-pose-estimation-3d-0001.onnx')
k_model = onnx_to_keras(onnx_model=onnx_model, input_names=['data'], change_ordering=True)

shutil.rmtree('/root/open_model_zoo/tools/downloader/public/human-pose-estimation-3d-0001/model/saved_model', ignore_errors=True)
tf.saved_model.save(k_model, '/root/open_model_zoo/tools/downloader/public/human-pose-estimation-3d-0001/model/saved_model')

# onnx 轉成 saveedmodel -方法2
#!onnx-tf convert -i /root/open_model_zoo/tools/downloader/public/human-pose-estimation-3d-0001/human-pose-estimation-3d-0001.onnx -o /root/open_model_zoo/tools/downloader/public/human-pose-estimation-3d-0001/model/saved_model

In [ ]:
#重啟環境 解決tensorflow版本問題
import os
os.kill(os.getpid(), 9)

In [ ]:
!pip install tensorflow==2.2.0

In [ ]:
# Tensorflow Lite 轉換
# Tensorflow 2.1.0 =>  Aborted (core dumped)
# Tensorflow 2.2.0 => Run => resolved reporter, Segmentation fault
# Tensorflow 2.5.0 => Run => didn't find op for builtin opcode "CONV" version 5
import tensorflow as tf

import numpy as np
def representative_dataset_gen():
    for _ in range(50):
        yield [np.random.uniform(0.0, 1.0, size=(1, 3, 256, 448)).astype(np.float32)] #需要輸入端大小

# Weight Quantization - Input/Output=float32
converter = tf.lite.TFLiteConverter.from_saved_model('/root/open_model_zoo/tools/downloader/public/human-pose-estimation-3d-0001/model/saved_model')
#converter = tf.compat.v1.lite.TFLiteConverter.from_saved_model('/root/open_model_zoo/tools/downloader/public/human-pose-estimation-3d-0001/model/saved_model')
#converter.optimizations = [tf.lite.Optimize.OPTIMIZE_FOR_SIZE]
#converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS_INT8]
#converter.inference_input_type  = tf.uint8
#converter.inference_output_type = tf.uint8
#converter.representative_dataset = representative_dataset_gen
tflite_quant_model = converter.convert()
with open('/root/open_model_zoo/tools/downloader/public/human-pose-estimation-3d-0001/model/human_pose_estimation_3d_0001_256x448_weight_quant.tflite', 'wb') as w:
    w.write(tflite_quant_model)
print("Weight Quantization complete! - human_pose_estimation_3d_0001_256x448_weight_quant.tflite")
#文章後續有介紹 整數量化 / 全整數量化 / 半浮點量化用法
#若要使用 tpu 請使用 edgetpu_compiler 再次生成 **_tpu.tflite

Weight Quantization complete! - human_pose_estimation_3d_0001_256x448_weight_quant.tflite


## 7.**MediaPipe 應用方式 (TensorFlow Lite Inverter)**

[@PINTO 大神](https://zenn.dev/pinto0309/articles/9d316860f8d418)



### **(1) Example : 以 BlazeFace(.tflite) 介紹**

In [ ]:
# 還原模組跟版本有很大關係
%tensorflow_version 1.x

TensorFlow 1.x selected.


In [ ]:
#重啟環境 解決tensorflow版本問題
import os
os.kill(os.getpid(), 9)

In [ ]:
#重新安裝 Tensorflow
!pip uninstall tensorflow -y
!pip install tensorflow==2.1.0

In [ ]:
#確認 Tensorflow 版本
import tensorflow
print(tensorflow.__version__)

In [ ]:
# 安裝 Flat Buffers (FlatBuffers是一個高效的跨平台序列化庫)
%cd /root
!git clone https://github.com/google/flatbuffers.git # flatbuffers ? what??

%cd /root/flatbuffers
#!git checkout v1.11.0

%cd /root/flatbuffers
!cmake -G "Unix Makefiles"
!make

In [ ]:
# Download Model
#!link google driver
from google.colab import drive
import pandas as pd
drive.mount('/content/gdrive')

# copy openvino file
!cp /content/gdrive/MyDrive/"Colab DataSet"/"OpenVINO Model"/schema_TF1x15_5.fbs /root/flatbuffers/schema.fbs
!cp /content/gdrive/MyDrive/"Colab DataSet"/"OpenVINO Model"/BlazeFace/face_detection_front.tflite /root/flatbuffers/face_detection_front.tflite


In [ ]:
# API
import os
import numpy as np
import json
import tensorflow as tf
import shutil
from pathlib import Path

# API config
%cd /root/flatbuffers/

os.environ['CUDA_VISIBLE_DEVICES'] = '0'
schema = "schema.fbs"
binary = "./flatc"
model_path = "palm_detection.tflite"
output_pb_path = "palm_detection.pb"
output_savedmodel_path = "saved_model"
model_json_path = "palm_detection.json"
num_tensors = 256 #須查看 Netron => Output's location +1
output_node_names = ['classificators', 'regressors']

# API
# 產生架構檔(JSON)
def gen_model_json():
  cmd = (binary + " -t --strict-json --defaults-json -o . {schema} -- {input}".format(input=model_path, schema=schema))
  print("output json command =", cmd)
  os.system(cmd)

# 解析架構
def parse_json():
    j = json.load(open(model_json_path))
    op_types = [v['builtin_code'] for v in j['operator_codes']]
    #print('op types:', op_types)
    ops = j['subgraphs'][0]['operators'] # subgraphs => 分為 tensors / inputs / outputs / operators
    # print('num of ops:', len(ops))
    return ops, op_types


def make_graph(ops, op_types, interpreter):
    tensors = {}
    input_details = interpreter.get_input_details()
    output_details = interpreter.get_output_details()
    # print(input_details)
    for input_detail in input_details:
        tensors[input_detail['index']] = tf.compat.v1.placeholder(
            dtype=input_detail['dtype'],
            shape=input_detail['shape'],
            name=input_detail['name'])

    for index, op in enumerate(ops):
        print('op: ', op)
        op_type = op_types[op['opcode_index']]
        if op_type == 'CONV_2D':
            input_tensor = tensors[op['inputs'][0]]
            weights_detail = interpreter._get_tensor_details(op['inputs'][1])
            bias_detail = interpreter._get_tensor_details(op['inputs'][2])
            output_detail = interpreter._get_tensor_details(op['outputs'][0])
            # print('weights_detail: ', weights_detail)
            # print('bias_detail: ', bias_detail)
            # print('output_detail: ', output_detail)
            weights_array = interpreter.get_tensor(weights_detail['index'])
            weights_array = np.transpose(weights_array, (1, 2, 3, 0))
            bias_array = interpreter.get_tensor(bias_detail['index'])
            weights = tf.compat.v1.Variable(weights_array, name=weights_detail['name'])
            bias = tf.compat.v1.Variable(bias_array, name=bias_detail['name'])
            options = op['builtin_options']
            output_tensor = tf.compat.v1.nn.conv2d(
                input_tensor,
                weights,
                strides=[1, options['stride_h'], options['stride_w'], 1],
                padding=options['padding'],
                dilations=[
                    1, options['dilation_h_factor'],
                    options['dilation_w_factor'], 1
                ],
                name=output_detail['name'] + '/conv2d')
            output_tensor = tf.compat.v1.add(
                output_tensor, bias, name=output_detail['name'])
            tensors[output_detail['index']] = output_tensor
        elif op_type == 'DEPTHWISE_CONV_2D':
            input_tensor = tensors[op['inputs'][0]]
            weights_detail = interpreter._get_tensor_details(op['inputs'][1])
            bias_detail = interpreter._get_tensor_details(op['inputs'][2])
            output_detail = interpreter._get_tensor_details(op['outputs'][0])
            # print('weights_detail: ', weights_detail)
            # print('bias_detail: ', bias_detail)
            # print('output_detail: ', output_detail)
            weights_array = interpreter.get_tensor(weights_detail['index'])
            weights_array = np.transpose(weights_array, (1, 2, 3, 0))
            bias_array = interpreter.get_tensor(bias_detail['index'])
            weights = tf.compat.v1.Variable(weights_array, name=weights_detail['name'])
            bias = tf.compat.v1.Variable(bias_array, name=bias_detail['name'])
            options = op['builtin_options']
            output_tensor = tf.compat.v1.nn.depthwise_conv2d(
                input_tensor,
                weights,
                strides=[1, options['stride_h'], options['stride_w'], 1],
                padding=options['padding'],
                # dilations=[
                #     1, options['dilation_h_factor'],
                #     options['dilation_w_factor'], 1
                # ],
                name=output_detail['name'] + '/depthwise_conv2d')
            output_tensor = tf.compat.v1.add(
                output_tensor, bias, name=output_detail['name'])
            tensors[output_detail['index']] = output_tensor
        elif op_type == 'MAX_POOL_2D':
            input_tensor = tensors[op['inputs'][0]]
            output_detail = interpreter._get_tensor_details(op['outputs'][0])
            options = op['builtin_options']
            output_tensor = tf.compat.v1.nn.max_pool(
                input_tensor,
                ksize=[
                    1, options['filter_height'], options['filter_width'], 1
                ],
                strides=[1, options['stride_h'], options['stride_w'], 1],
                padding=options['padding'],
                name=output_detail['name'])
            tensors[output_detail['index']] = output_tensor
        elif op_type == 'PAD':
            input_tensor = tensors[op['inputs'][0]]
            output_detail = interpreter._get_tensor_details(op['outputs'][0])
            paddings_detail = interpreter._get_tensor_details(op['inputs'][1])
            print('output_detail:', output_detail)
            print('paddings_detail:', paddings_detail)
            paddings_array = interpreter.get_tensor(paddings_detail['index'])
            paddings = tf.compat.v1.Variable(paddings_array, name=paddings_detail['name'])
            paddings = tf.constant(paddings_array)
            print('paddings:',paddings)
            output_tensor = tf.compat.v1.pad(
                input_tensor, paddings, name=output_detail['name'])
            tensors[output_detail['index']] = output_tensor
        elif op_type == 'RELU':
            output_detail = interpreter._get_tensor_details(op['outputs'][0])
            input_tensor = tensors[op['inputs'][0]]
            output_tensor = tf.compat.v1.nn.relu(
                input_tensor, name=output_detail['name'])
            tensors[output_detail['index']] = output_tensor
        elif op_type == 'PRELU':
            output_detail = interpreter._get_tensor_details(op['outputs'][0])
            input_tensor = tensors[op['inputs'][0]]
            alpha_detail = interpreter._get_tensor_details(op['inputs'][1])
            alpha_array = interpreter.get_tensor(alpha_detail['index'])
            with tf.compat.v1.variable_scope(name_or_scope=output_detail['name']):
                alphas = tf.compat.v1.Variable(alpha_array, name=alpha_detail['name'])
                output_tensor = tf.compat.v1.maximum(alphas * input_tensor, input_tensor)
            tensors[output_detail['index']] = output_tensor
        elif op_type == 'RESHAPE':
            input_tensor = tensors[op['inputs'][0]]
            output_detail = interpreter._get_tensor_details(op['outputs'][0])
            options = op['builtin_options']
            output_tensor = tf.compat.v1.reshape(
                input_tensor, options['new_shape'], name=output_detail['name'])
            tensors[output_detail['index']] = output_tensor

        elif op_type == 'RESIZE_BILINEAR':
            output_detail = interpreter._get_tensor_details(op['outputs'][0])
            input_tensor = tensors[op['inputs'][0]]
            size_detail = interpreter._get_tensor_details(op['inputs'][1])
            size = interpreter.get_tensor(size_detail['index'])
            size_height = size[0]
            size_width  = size[1]
            tensors[output_detail['index']] = tf.image.resize(input_tensor, [size_height, size_width], method='bilinear')
            #def upsampling2d_bilinear(x, size_height, size_width):
            #  return  tf.image.resize(x, [size_height, size_width], method='bilinear')
            #output_tensor = tf.keras.layers.Lambda(upsampling2d_bilinear, arguments={'size_height': size_height, 'size_width': size_width})(input_tensor)
            #tensors[output_detail['index']] = output_tensor

        elif op_type == 'ADD':
            output_detail = interpreter._get_tensor_details(op['outputs'][0])
            input_tensor_0 = tensors[op['inputs'][0]]
            input_tensor_1 = tensors[op['inputs'][1]]
            output_tensor = tf.compat.v1.add(input_tensor_0, input_tensor_1, name=output_detail['name'])
            tensors[output_detail['index']] = output_tensor

        elif op_type == 'CONCATENATION':
            output_detail = interpreter._get_tensor_details(op['outputs'][0])
            input_tensor_0 = tensors[op['inputs'][0]]
            input_tensor_1 = tensors[op['inputs'][1]]
            options = op['builtin_options']
            output_tensor = tf.compat.v1.concat([input_tensor_0, input_tensor_1],
                                      options['axis'],
                                      name=output_detail['name'])
            tensors[output_detail['index']] = output_tensor
        elif op_type == 'AVERAGE_POOL_2D':
            output_detail = interpreter._get_tensor_details(op['outputs'][0])
            input_tensor = tensors[op['inputs'][0]]
            options = op['builtin_options']
            pool_size = [options['filter_height'], options['filter_width']]
            strides = [options['stride_h'], options['stride_w']]
            padding = options['padding']
            output_tensor = tf.keras.layers.AveragePooling2D(pool_size=pool_size,
                                      strides=strides,
                                      padding=padding,
                                      name=output_detail['name'])(input_tensor)
            tensors[output_detail['index']] = output_tensor
        elif op_type == 'SOFTMAX':
            output_detail = interpreter._get_tensor_details(op['outputs'][0])
            input_tensor = tensors[op['inputs'][0]]
            output_tensor = tf.compat.v1.nn.softmax(input_tensor, name=output_detail['name'])
        elif op_type == 'DEQUANTIZE':
            output_detail = interpreter._get_tensor_details(op['outputs'][0])
            weights_detail = interpreter._get_tensor_details(op['inputs'][0])
            weights = interpreter.get_tensor(weights_detail['index'])
            output_tensor = weights.astype(np.float32)
            tensors[output_detail['index']] = output_tensor
        else:
            raise ValueError(op_type)


#-----------------------------------------------------------------------------------------------------------------------------------------------
# MAIN
#-----------------------------------------------------------------------------------------------------------------------------------------------

# 將 tf lite 重新拆解回 savedmodel
tf.compat.v1.disable_eager_execution()
gen_model_json()
ops, op_types = parse_json()
interpreter = tf.lite.Interpreter(model_path)
interpreter.allocate_tensors()
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()
print(input_details)
print(output_details)

make_graph(ops, op_types, interpreter) #error

config = tf.compat.v1.ConfigProto()
config.gpu_options.allow_growth = True
graph = tf.compat.v1.get_default_graph()

with tf.compat.v1.Session(config=config, graph=graph) as sess:
  sess.run(tf.compat.v1.global_variables_initializer())
  graph_def = tf.compat.v1.graph_util.convert_variables_to_constants(sess=sess,input_graph_def=graph.as_graph_def(),output_node_names=output_node_names)

  with tf.io.gfile.GFile(output_pb_path, 'wb') as f:
    f.write(graph_def.SerializeToString())

  shutil.rmtree('saved_model', ignore_errors=True)
  tf.compat.v1.saved_model.simple_save(sess,
            output_savedmodel_path,
            inputs={'input': graph.get_tensor_by_name('input:0')},
            outputs={
                #'MobilenetV1/Predictions/Reshape_1' : graph.get_tensor_by_name('MobilenetV1/Predictions/Reshape_1:0')
                'classificators': graph.get_tensor_by_name('classificators:0'),
                'regressors': graph.get_tensor_by_name('regressors:0')
            })
#-----------------------------------------------------------------------------------------------------------------------------------------------

In [ ]:
# 檢視 op_types 型態狀況
# 如果錯誤的話 (都是ADD) 則須重新調用匹配 Tesnorflow 與 schema.fbs 版本

#!./flatc -t --strict-json --defaults-json -o . schema.fbs -- mobilenet_v1_1.0_224.tflite
ops, op_types = parse_json()
print(op_types)

# 可檢視之項目 "version" "operator_codes" "subgraphs" "description" "buffers" "metadata_buffer"
'''
j = json.load(open(model_json_path))
j['version']
j['operator_codes']
j['subgraphs']
j['description']
j['buffers']
j['metadata_buffer']
'''

# 可以進一步細分 subgraphs 之項目 "tensors" "inputs" "outputs" "operators"
'''
j['subgraphs'][0]['tensors']
j['subgraphs'][0]['inputs']
j['subgraphs'][0]['outputs']
j['subgraphs'][0]['operators']
'''

In [ ]:
# Tensorflow Lite 轉換 (浮點)
import tensorflow as tf
import numpy as np
converter = tf.compat.v1.lite.TFLiteConverter.from_saved_model('/root/flatbuffers/saved_model')
#converter.optimizations = [tf.compat.v1.lite.Optimize.OPTIMIZE_FOR_SIZE]
#converter.target_spec.supported_ops = [tf.compat.v1.lite.OpsSet.TFLITE_BUILTINS, tf.compat.v1.lite.OpsSet.SELECT_TF_OPS]
tflite_quant_model = converter.convert()
with open('/root/palm_detection_float_1.tflite', 'wb') as w:
    w.write(tflite_quant_model)
print("Weight Quantization complete!")

In [ ]:
# Tensorflow Lite 轉換 (全整數) 
# 可以使用但會出現 error : undefined identifier: 訊息
import tensorflow as tf
import numpy as np
def representative_dataset_gen():
    for _ in range(50):
        yield [np.random.uniform(0.0, 1.0, size=(1, 128, 128, 3)).astype(np.float32)] #需要輸入端大小

converter = tf.compat.v1.lite.TFLiteConverter.from_saved_model('/root/flatbuffers/saved_model')
#converter.optimizations = [tf.compat.v1.lite.Optimize.OPTIMIZE_FOR_SIZE]
converter.target_spec.supported_ops = [tf.compat.v1.lite.OpsSet.TFLITE_BUILTINS_INT8]
converter.inference_input_type  = tf.compat.v1.uint8
converter.inference_output_type = tf.compat.v1.uint8
converter.representative_dataset = representative_dataset_gen
tflite_quant_model = converter.convert()
with open('/root/palm_detection_uint8.tflite', 'wb') as w:
    w.write(tflite_quant_model)
print("Weight Quantization complete!")

### **(2) Testing**

In [ ]:
# 可用 keras 描述
import os
import numpy as np
import json
import tensorflow as tf
import shutil
from pathlib import Path

# API config
%cd /root/flatbuffers/
!rm -r /root/flatbuffers/saved_model

os.environ['CUDA_VISIBLE_DEVICES'] = '0'
schema = "schema.fbs"
binary = "./flatc"
model_path = "posenet.tflite"
output_pb_path = "posenet.pb"
output_savedmodel_path = "saved_model"
model_json_path = "posenet.json"
output_node_names = ['MobilenetV1/heatmap_2/BiasAdd','MobilenetV1/offset_2/BiasAdd','MobilenetV1/displacement_fwd_2/BiasAdd','MobilenetV1/displacement_bwd_2/BiasAdd']

# API
# 產生架構檔(JSON)
def gen_model_json():
  cmd = (binary + " -t --strict-json --defaults-json -o . {schema} -- {input}".format(input=model_path, schema=schema))
  print("output json command =", cmd)
  os.system(cmd)

# 解析架構
def parse_json():
    j = json.load(open(model_json_path))
    op_types = [v['builtin_code'] for v in j['operator_codes']]
    #print('op types:', op_types)
    ops = j['subgraphs'][0]['operators'] # subgraphs => 分為 tensors / inputs / outputs / operators
    print('num of ops:', len(ops))
    return ops, op_types

def make_graph(ops, op_types, interpreter):
    tensors = {}
    input_details = interpreter.get_input_details()
    output_details = interpreter.get_output_details()
    # print(input_details)
    for input_detail in input_details:
        tensors[input_detail['index']] = tf.compat.v1.placeholder(
            dtype=input_detail['dtype'],
            shape=input_detail['shape'],
            name=input_detail['name'])

    for index, op in enumerate(ops):
        print('op: ', op)
        op_type = op_types[op['opcode_index']]
        if op_type == 'CONV_2D':
            input_tensor = tensors[op['inputs'][0]]
            weights_detail = interpreter._get_tensor_details(op['inputs'][1])
            bias_detail = interpreter._get_tensor_details(op['inputs'][2])
            output_detail = interpreter._get_tensor_details(op['outputs'][0])
            # print('weights_detail: ', weights_detail)
            # print('bias_detail: ', bias_detail)
            # print('output_detail: ', output_detail)
            weights_array = interpreter.get_tensor(weights_detail['index'])
            weights_array = np.transpose(weights_array, (1, 2, 3, 0))
            bias_array = interpreter.get_tensor(bias_detail['index'])
            weights = tf.compat.v1.Variable(weights_array, name=weights_detail['name'])
            bias = tf.compat.v1.Variable(bias_array, name=bias_detail['name'])
            options = op['builtin_options']
            output_tensor = tf.compat.v1.nn.conv2d(
                input_tensor,
                weights,
                strides=[1, options['stride_h'], options['stride_w'], 1],
                padding=options['padding'],
                dilations=[
                    1, options['dilation_h_factor'],
                    options['dilation_w_factor'], 1
                ],
                name=output_detail['name'] + '/conv2d')
            #output_tensor = tf.compat.v1.add(output_tensor, bias, name=output_detail['name'])
            tensors[output_detail['index']] = output_tensor
        elif op_type == 'DEPTHWISE_CONV_2D':
            input_tensor = tensors[op['inputs'][0]]
            weights_detail = interpreter._get_tensor_details(op['inputs'][1])
            bias_detail = interpreter._get_tensor_details(op['inputs'][2])
            output_detail = interpreter._get_tensor_details(op['outputs'][0])
            # print('weights_detail: ', weights_detail)
            # print('bias_detail: ', bias_detail)
            # print('output_detail: ', output_detail)
            weights_array = interpreter.get_tensor(weights_detail['index'])
            weights_array = np.transpose(weights_array, (1, 2, 3, 0))
            bias_array = interpreter.get_tensor(bias_detail['index'])
            weights = tf.compat.v1.Variable(weights_array, name=weights_detail['name'])
            bias = tf.compat.v1.Variable(bias_array, name=bias_detail['name'])
            options = op['builtin_options']
            output_tensor = tf.compat.v1.nn.depthwise_conv2d(
                input_tensor,
                weights,
                strides=[1, options['stride_h'], options['stride_w'], 1],
                padding=options['padding'],
                # dilations=[
                #     1, options['dilation_h_factor'],
                #     options['dilation_w_factor'], 1
                # ],
                name=output_detail['name'] + '/depthwise_conv2d')
            #output_tensor = tf.compat.v1.add(output_tensor, bias, name=output_detail['name'])
            tensors[output_detail['index']] = output_tensor
        elif op_type == 'MAX_POOL_2D':
            input_tensor = tensors[op['inputs'][0]]
            output_detail = interpreter._get_tensor_details(op['outputs'][0])
            options = op['builtin_options']
            output_tensor = tf.compat.v1.nn.max_pool(
                input_tensor,
                ksize=[
                    1, options['filter_height'], options['filter_width'], 1
                ],
                strides=[1, options['stride_h'], options['stride_w'], 1],
                padding=options['padding'],
                name=output_detail['name'])
            tensors[output_detail['index']] = output_tensor
        elif op_type == 'PAD':
            input_tensor = tensors[op['inputs'][0]]
            output_detail = interpreter._get_tensor_details(op['outputs'][0])
            paddings_detail = interpreter._get_tensor_details(op['inputs'][1])
            print('output_detail:', output_detail)
            print('paddings_detail:', paddings_detail)
            paddings_array = interpreter.get_tensor(paddings_detail['index'])
            paddings = tf.compat.v1.Variable(paddings_array, name=paddings_detail['name'])
            paddings = tf.constant(paddings_array)
            print('paddings:',paddings)
            output_tensor = tf.compat.v1.pad(
                input_tensor, paddings, name=output_detail['name'])
            tensors[output_detail['index']] = output_tensor
        elif op_type == 'RELU':
            output_detail = interpreter._get_tensor_details(op['outputs'][0])
            input_tensor = tensors[op['inputs'][0]]
            output_tensor = tf.compat.v1.nn.relu(
                input_tensor, name=output_detail['name'])
            tensors[output_detail['index']] = output_tensor
        elif op_type == 'PRELU':
            output_detail = interpreter._get_tensor_details(op['outputs'][0])
            input_tensor = tensors[op['inputs'][0]]
            alpha_detail = interpreter._get_tensor_details(op['inputs'][1])
            alpha_array = interpreter.get_tensor(alpha_detail['index'])
            with tf.compat.v1.variable_scope(name_or_scope=output_detail['name']):
                alphas = tf.compat.v1.Variable(alpha_array, name=alpha_detail['name'])
                output_tensor = tf.compat.v1.maximum(alphas * input_tensor, input_tensor)
            tensors[output_detail['index']] = output_tensor
        elif op_type == 'RESHAPE':
            input_tensor = tensors[op['inputs'][0]]
            output_detail = interpreter._get_tensor_details(op['outputs'][0])
            options = op['builtin_options']
            output_tensor = tf.compat.v1.reshape(
                input_tensor, options['new_shape'], name=output_detail['name'])
            tensors[output_detail['index']] = output_tensor

        elif op_type == 'RESIZE_BILINEAR':
            output_detail = interpreter._get_tensor_details(op['outputs'][0])
            input_tensor = tensors[op['inputs'][0]]
            size_detail = interpreter._get_tensor_details(op['inputs'][1])
            size = interpreter.get_tensor(size_detail['index'])
            size_height = size[0]
            size_width  = size[1]
            tensors[output_detail['index']] = tf.image.resize(input_tensor, [size_height, size_width], method='bilinear')
            #def upsampling2d_bilinear(x, size_height, size_width):
            #  return  tf.image.resize(x, [size_height, size_width], method='bilinear')
            #output_tensor = tf.keras.layers.Lambda(upsampling2d_bilinear, arguments={'size_height': size_height, 'size_width': size_width})(input_tensor)
            #tensors[output_detail['index']] = output_tensor

        elif op_type == 'ADD':
            output_detail = interpreter._get_tensor_details(op['outputs'][0])
            input_tensor_0 = tensors[op['inputs'][0]]
            input_tensor_1 = tensors[op['inputs'][1]]
            output_tensor = tf.compat.v1.add(input_tensor_0, input_tensor_1, name=output_detail['name'])
            tensors[output_detail['index']] = output_tensor

        elif op_type == 'CONCATENATION':
            output_detail = interpreter._get_tensor_details(op['outputs'][0])
            input_tensor_0 = tensors[op['inputs'][0]]
            input_tensor_1 = tensors[op['inputs'][1]]
            options = op['builtin_options']
            output_tensor = tf.compat.v1.concat([input_tensor_0, input_tensor_1],
                                      options['axis'],
                                      name=output_detail['name'])
            tensors[output_detail['index']] = output_tensor
        elif op_type == 'AVERAGE_POOL_2D':
            output_detail = interpreter._get_tensor_details(op['outputs'][0])
            input_tensor = tensors[op['inputs'][0]]
            options = op['builtin_options']
            pool_size = [options['filter_height'], options['filter_width']]
            strides = [options['stride_h'], options['stride_w']]
            padding = options['padding']
            output_tensor = tf.keras.layers.AveragePooling2D(pool_size=pool_size,
                                      strides=strides,
                                      padding=padding,
                                      name=output_detail['name'])(input_tensor)
            tensors[output_detail['index']] = output_tensor
        elif op_type == 'SOFTMAX':
            output_detail = interpreter._get_tensor_details(op['outputs'][0])
            input_tensor = tensors[op['inputs'][0]]
            output_tensor = tf.compat.v1.nn.softmax(input_tensor, name=output_detail['name'])
        elif op_type == 'DEQUANTIZE':
            output_detail = interpreter._get_tensor_details(op['outputs'][0])
            weights_detail = interpreter._get_tensor_details(op['inputs'][0])
            weights = interpreter.get_tensor(weights_detail['index'])
            output_tensor = weights.astype(np.float32)
            tensors[output_detail['index']] = output_tensor
        else:
            raise ValueError(op_type)

#-----------------------------------------------------------------------------------------------------------------------------------------------
# MAIN
#-----------------------------------------------------------------------------------------------------------------------------------------------

# 將 tf lite 重新拆解回 savedmodel
tf.compat.v1.disable_eager_execution()
gen_model_json()
ops, op_types = parse_json()
interpreter = tf.lite.Interpreter(model_path)
interpreter.allocate_tensors()
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()
print(input_details)
print(output_details)

make_graph(ops, op_types, interpreter) #error

config = tf.compat.v1.ConfigProto()
config.gpu_options.allow_growth = True
graph = tf.compat.v1.get_default_graph()

with tf.compat.v1.Session(config=config, graph=graph) as sess:
  sess.run(tf.compat.v1.global_variables_initializer())
  graph_def = tf.compat.v1.graph_util.convert_variables_to_constants(sess=sess,input_graph_def=graph.as_graph_def(),output_node_names=output_node_names)

  with tf.io.gfile.GFile(output_pb_path, 'wb') as f:
    f.write(graph_def.SerializeToString())

  shutil.rmtree('saved_model', ignore_errors=True)
  tf.compat.v1.saved_model.simple_save(sess,
            output_savedmodel_path,
            inputs={'sub_2': graph.get_tensor_by_name('sub_2:0')},
            outputs={
                'MobilenetV1/heatmap_2/BiasAdd' : graph.get_tensor_by_name('MobilenetV1/heatmap_2/BiasAdd:0'),
                'MobilenetV1/offset_2/BiasAdd' : graph.get_tensor_by_name('MobilenetV1/offset_2/BiasAdd:0'),
                'MobilenetV1/displacement_fwd_2/BiasAdd' : graph.get_tensor_by_name('MobilenetV1/displacement_fwd_2/BiasAdd:0'),
                'MobilenetV1/displacement_bwd_2/BiasAdd' : graph.get_tensor_by_name('MobilenetV1/displacement_bwd_2/BiasAdd:0')
            })
  
#-----------------------------------------------------------------------------------------------------------------------------------------------

In [ ]:
ops, op_types = parse_json()
print(op_types)

['CONV_2D', 'DEPTHWISE_CONV_2D']


In [ ]:
# Tensorflow Lite 轉換 (浮點)
import tensorflow as tf
import numpy as np
converter = tf.compat.v1.lite.TFLiteConverter.from_saved_model('/root/flatbuffers/saved_model')
#converter.optimizations = [tf.compat.v1.lite.Optimize.OPTIMIZE_FOR_SIZE]
#converter.target_spec.supported_ops = [tf.compat.v1.lite.OpsSet.TFLITE_BUILTINS, tf.compat.v1.lite.OpsSet.SELECT_TF_OPS]
tflite_quant_model = converter.convert()
with open('/root/posenet_new.tflite', 'wb') as w:
    w.write(tflite_quant_model)
print("Weight Quantization complete!")

**Problem　１　：　tf2 converter**

UnliftableError: A SavedModel signature needs an input for each placeholder the signature's outputs use.

(solve) : 儲存模組的代碼寫錯 

"---------------------------------------------------------------------------------------------------------------------------------------------"

**Problem　2　：　tf1 converter**

ConverterError: "Graph does not contain node Input.

(solve) : 儲存模組的代碼寫錯 

"---------------------------------------------------------------------------------------------------------------------------------------------"

**Problem　3　：　tf lite runtime**

ERROR: Didn't find op for builtin opcode 'CONV_2D' version '5'.

(solve) : 降低 Tensorflow 版本至 2.1.0, 但衍伸問題4

"---------------------------------------------------------------------------------------------------------------------------------------------"

**Problem　4　：　tf lite runtime**

Error : ERROR: Node number 164 (TfLiteNnapiDelegate) failed to invoke.

(solve) : num_tensors 數量設錯

"---------------------------------------------------------------------------------------------------------------------------------------------"

**Problem　5　：　tf lite runtime**

Error : Netron 架構多了 Dequantize 這層

(solve) : 不進行 converter.optimizations 轉換